In [1]:
from typing import Callable, Type, TypeVar, Generic

T = TypeVar("T")


class Callback(Generic[T]):
    """回調函數類別，支援參數類型檢查"""

    callback_type: Type[T]
    callbacks: dict[str, Callable[..., T]] = {}

    def __init__(self, callback_type: Type[T]):
        """初始化回調函數類別

        Args:
            callback_type (Type[T]): 回調函數的返回值類型
        """
        self.callback_type = callback_type
        self.callbacks: dict[str, Callable[..., T]] = {}

    def register(self, event_name: str, handler: Callable[..., T] | None) -> None:
        """註冊回調函數

        Args:
            event_name: 事件名稱
            handler: 回調函數，必須返回指定的類型
        """
        if not callable(handler):
            raise TypeError("Handler must be a callable function")
        self.callbacks[event_name] = handler

    def unregister(self, event_name: str) -> None:
        """取消註冊回調函數

        Args:
            event_name: 要取消的事件名稱
        """
        self.callbacks.pop(event_name, None)

    def trigger(self, event_name: str, *args, **kwargs) -> T:
        """觸發回調函數

        Args:
            event_name: 要觸發的事件名稱
            *args: 位置參數
            **kwargs: 關鍵字參數

        Returns:
            回調函數的返回值，類型為初始化時指定的類型
        """
        if event_name in self.callbacks:
            result = self.callbacks[event_name](*args, **kwargs)
            if hasattr(self.callback_type, "__origin__"):
                # 處理泛型類型如 tuple[int, float]
                if not isinstance(result, self.callback_type.__origin__):
                    raise TypeError(
                        f"Return value must be of type {self.callback_type}"
                    )
                # 檢查泛型參數類型
                if len(result) != len(self.callback_type.__args__):
                    raise TypeError(
                        f"Return value must contain {len(self.callback_type.__args__)} elements"
                    )
                for val, expected_type in zip(result, self.callback_type.__args__):
                    if not isinstance(val, expected_type):
                        raise TypeError(
                            f"Return value elements must be of type {self.callback_type}"
                        )
            else:
                # 處理一般類型
                if not isinstance(result, self.callback_type):
                    raise TypeError(
                        f"Return value must be of type {self.callback_type}"
                    )
            return result
        else:
            raise KeyError(f"No callback found for event {event_name}")

In [11]:
dir(tuple)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index']

In [12]:
dir(tuple[int, float])

['__add__',
 '__args__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mro_entries__',
 '__mul__',
 '__ne__',
 '__new__',
 '__origin__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index']

In [1]:
print(tuple.__name__)
print((tuple[int, float]).__origin__)

tuple
<class 'tuple'>


In [6]:
on_click = Callback(tuple[int, float])

on_click.register("click", lambda: 1)
on_click.register("click", lambda: "1")
on_click.trigger("click")

TypeError: isinstance() argument 2 cannot be a parameterized generic